# Drift Detection Infrastructure

In dieser Übung setzen wir den Monitoring Dienst auf und lernen diesen kennen. Wird verwenden dazu [Prometheus](https://prometheus.io/docs/introduction/overview/) mit dem [Statsd Exporter](https://github.com/prometheus/statsd_exporter) und [Grafana](https://grafana.com/). 

Diese Tools werden bereits von vielen Organisationen für das Sammeln von Metriken, das Darstellen deren Verlauf und für die Alarmierung verwendet.

# Vorbereitung

Speichere alle deine offenen Notebooks. Stoppe alle Container, indem uu Dich ins Top-Level Verzeichnis der Übungen begibst (dort, wo das File `docker-compose.yml` liegt) und `docker compose down` ausführst. Wenn Du die Container im Vordergrund laufen hast, kannst Du in diesem Terminal die Prozesse mit `ctrl-c` stoppen.

Kommentiere im Top-Level Compose File `docker-compose.yml` den Monitoring Dienst ein.

Starte alle Container wieder.

# Übungen

## Architektur

Wir werden die folgenden Architektur aufsetzen:

![monitoring_01.png](monitoring_01.png)

Dabei ist es wichtig, den Einsatzzweck der einzelnen Komponenten zu verstehen.
* **Prometheus**: Sammelt und speichert Datenpunkte und bietet mit PromQL eine eigene Sprache, um diese zu analysieren. Prometheus pullt Metriken immer aktiv, was schön ist für Services, welche dann ihre Metriken einfach auf einer Webseite in dem von Prometheus vorgegebenen Format anbieten können.
* **Grafana**: Holt Daten von verschiedenen Quellen und stellt diese grafisch dar. Kann Alarme aussenden.
* **Unsere Batch- und Streaming Pipelines**: Berechnen und pushen Metriken aktiv.
* **Statsd Exporter**: Nimmt Metriken im Statsd Format entgegen und stellt für Prometheus einen entsprechende Endpoint zum Pullen zur Verfügung.

Das Statsd Format ist ein gängiges Format zum Loggen von Metriken. Der Statsd_Exporter übersetzt von diesem Format in ein für Prometheus verständliches Format.

Wieso kann auf die Funktionalität des Statsd_Exporters nicht verzichtet werden?

Hätten wir nur Streaming-Pipelines, könnte theoretisch auf die Funktionalität des Statsd_Exporters verzichtet werden. Die Pipelines könnten selber für das Polling von Prometheus einen entsprechenden Endpoint zur Verfügung stellen, da sie ja konstant laufen.

Bei Batch-Pipelines, welche periodisch gestartet werden, durchlaufen und sich dann beenden, braucht es die Funktionalität des Statsd_Exporters hingegen zwingend.

----

## Statsd Service

Schau dir das compose file der Monitoring Dienste an. Via welchen Port kannst du von ausserhalb Docker auf Statsd zugreifen?

Von ausserhalb Docker kannst du via den Port 9102 zugreifen.

----

Nun schau dir das Mapping File an. Welche vier Informationen geben wir mit, wenn unsere Pipeline eine Drift Metrik loggt?

Den Namen des Datensets, für welches wir den Drift berechnen (dataset_name), dessen Version (dataset_version), den Spaltennamen, um welchen es geht (column_name) sowie die Bezeichnung der Metrik, welche wir berechnen (metric_name).

----

Greife nun auf die Seite der Metriken zu, welche der Statsd_Exporter für Prometheus zur Verfügung stellt: [http://localhost:9102/metrics](http://localhost:9102/metrics). Du siehst bereits viel Text, ohne dass wir schon etwas geloggt hätten. Was könnte dieser bedeuten?

Es handelt sich um Metriken, welche der Statsd_Exporter über sich selber zur Verfügung stellt, damit er als Service überwacht werden kann.

----

Der Exporter liest jeden empfangenen String, und wenn dieser in eines der definierten Mappings passt, erscheint der (aktualisierte) Wert auf der `/metrics` Webseite und kann von Prometheus gepollt werden.

Wie muss ein gültiger String aussehen, der eine unserer Drift Metriken definiert?

Die `match` direktive aus dem Mappings File zeigt das Muster an mit vier Platzhalter an:

    match: "drift_metrics.*.*.*.*"
    
Für die `*` können beliebige Werte einfügen, z.B.

    drift_metrics.mushroom.v1.cap-diameter.wasserstein
    
Der eigentliche Wert wird, abgetrennt durch einen Doppelpunkt angehängt. Danach folgt, abgetrennt durch eine Pipe, der Typ der Metrik. Welche Typen es gibt, sehen wir später.

    drift_metrics.mushroom.v1.cap-diameter.wasserstein:42|g
    
Die vier `labels` werden für die Umformatierung ins Prometheus Format verwendet.

----

Versuche nun testweise, den Wert einer Metrik an den Statsd_Exporter zu schicken. Dazu musst du den obigen String via UDP an den Port 8125 schicken. Am einfachsten geht dies via [netcat](https://en.wikipedia.org/wiki/Netcat). Da der Port aber nicht nach ausserhalb des Docker-Netzwerkes gemappt ist (nur `expose` und nicht `ports` im compose file), und netcat möglicherweise nicht auf dem Host (dein Laptop oder codespaces) installiert ist, verwendest du ein Docker Image. Führe den folgenden Befehl aus:

    docker run -i --rm --network=production --name netcat_test subfuzion/netcat -u statsd 8125

Netcat wartet auf eine Eingabe (kein Prompt sichtbar), du pastest den String von oben und drückst Enter.

    drift_metrics.mushroom.v1.cap-diameter.wasserstein:42|g

Nun gehe noch einmal auf die Webseite [http://localhost:9102/metrics](http://localhost:9102/metrics). Die Metrik müsste nun mit dem Wert 42 sichtbar sein (wahrscheinlich oben an der Seite):

    drift_metrics{column_name="cap-diameter",dataset_name="mushroom",dataset_version="v1",metric_name="wasserstein"} 42
    
Wir sehen, dass der Exporter die Angaben aus dem Mappings File verwendet hat, um die erhaltene Metrik für Prometheus umzuformatieren.

Den noch laufenden netcat Container kannst Du mittels `docker stop netcat_test` stoppen.

## Prometheus Service

Schau Dir nun das Config File von Prometheus an. Was siehst du?

Es sind zwei Jobs definiert, Prometheus sammelt also Daten von zweien Services. Einerseits von sich selbst, und andererseits unsere Drift Metriken via Statsd. Eigentlich sollte noch eine dritte Quelle vorhanden sein, nämlich die Metriken des Statsd_Exporters selber. Diese haben wir der Einfachheit halber ausgelassen.

----

Öffne nun die Webseite von Prometheus: [http://localhost:9090/](http://localhost:9090/). Versuche, die vorher testweise geloggte Drift Metrik sowie Metriken zu Prometheus selber anzuzeigen.

Du kannst dazu im Query-Fenster beispielsweise `drift_metrics` eingeben und `Execute` drücken. Auto-Complete funktioniert auch.

----

Zeige nun noch via das Status Menu die konfigurierten Targets an. Du siehst die im Config File definierten Targets, `prometheus` und `dataset_metrics`.

## Grafana

Nun greife auf [Grafana](http://localhost:3000/) zu. Erstelle als erstes eine Data Source für Prometheus.

1. Klicke im Sidebar links auf `Connections -> Data sources` und dann auf `Add new data source` oben rechts.
1. Suche und wähle `Prometheus`
1. Setze unter Connections die `Prometheus server URL` auf `http://prometheus:9090`
1. Klicke ganz unten auf `Save & test`, es sollte eine grüne Erfolgsmeldung Meldung erscheinen

----

Als nächstes erstellst Du ein neues Dashboard und fügst diesem eine Visualisierung hinzu. Baue Dir eine Visualisierung, welche die Drift Metrik auf eine einfache Weise anzeigt. Du wirst noch nicht viel sehen können, denn du hast erst einen einzigen Datenpunkt, den vom obigen Test.

Du kannst das Dashboard später ausbauen, wenn wir mehr Daten haben.

1. `Dashboards -> New`
1. `Add visualization`
1. Prometheus auswählen
1. Unter `Metric` im `Select metric` dropdown `drift_metrics` wählen
1. Button `Run queries` betätigen

Im Panel sieht man nun einen kleinen Punkt und in der Legende eine lange Zeile. Dieses automatisch generierte Label ist etwas unleserlich. Unter `Options` können wir das anpassen: Legend = Custom, statt = `{{label_name}}` setzen wir `{{column_name}}`.

Nun noch rechts oben einen `Panel Title` setzen und Save klicken und Dashboard benennen.

----

Wenn Du möchtest, kannst du nun in Prometheus noch Grafana selber als Quelle hinzufügen, damit wir auch den Grafana Dienst überwachen können :-)

# Architekturübersicht

Unsere erweiterte Architektur sieht nun so aus.

![monitoring_02.png](monitoring_02.png)

----

**Bitte quittiere wiederum auf [Mentimeter](https://www.menti.com/alaxbnek73eu), dass du mit der Übung durch bist**.

Du kannst nun gleich mit der nächsten Übung weiterfahren (`19_Drift_Detection_Pipeline.ipynb`).